In [1]:
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv



In [2]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(list(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [3]:
HG = load_ABCDH_from_file("datasets/results_300_he.txt")



In [4]:
hL = hl.hLouvain(HG,hmod_type=hmod.linear, 
                                    delta_it = 0.0001, 
                                    delta_phase = 0.0001, 
                                    random_seed = 5673) 

In [5]:
alphas = []
d = 0
c = 0.2
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i)*(1-d))
    
print("alphas", alphas)
    
A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, 
                                                change_mode="communities", change_frequency=c)
qH = hL.combined_modularity(A, hmod.linear, 1)

print("alphas_out", alphas_out)

print("partition",A)
print("gH =",qH)

alphas [0.0, 0.8, 0.96, 0.992, 0.9984, 0.99968, 0.999936, 0.9999872, 0.99999744, 0.999999488, 0.9999998976, 0.99999997952, 0.999999995904, 0.9999999991808, 0.99999999983616, 0.999999999967232, 0.9999999999934464, 0.9999999999986893, 0.9999999999997379, 0.9999999999999476, 0.9999999999999896, 0.9999999999999979, 0.9999999999999996, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
alphas_out [0.0, 0.8]
partition [{'57', '74', '284', '297', '103', '276', '213', '179', '66', '238', '149', '43', '50', '190', '93'}, {'255', '177', '32', '63', '235', '82', '147', '114', '44', '290', '40', '180', '62', '283', '186', '167', '13', '162', '233', '4', '236'}, {'188', '20', '85', '222', '151', '95', '237', '12', '128', '155', '72', '88', '228', '116', '249', '229', '132', '210', '145', '143', '109', '97', '106', '46', '18', '247', '265', '288', '173', '223', '251', '78', '193', '152'}, {'168', '246', '100', '130', '217', '212', '113', '21', '287', '84', '118', '240', '245'}, {'1', '150', '153', '1

In [6]:
qH = hL.combined_modularity(A, hmod.linear, 1)
print(qH)
two_section_modularity = hL.combined_modularity(A, hmod.linear, 0)
print(two_section_modularity)

0.5286210742178985
0.45395965613812433


In [7]:
with open("Data/results_300_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
A_gt = [x for x in hmod.dict2part({str(i+1):gt[i] for i in range(len(gt))}) if len(x)>0]
gt_linear = hL.combined_modularity(A_gt, hmod.linear, 1)

print("ground-truth partition")
print(A_gt)
print("qh-gt-linear:",gt_linear)

def getAMI_ARI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ari = [d[str(i+1)] for i in range(len(HG.nodes))]
    return ARI(gt, A4ari), AMI(gt, A4ari)

ground-truth partition
[{'254', '65', '285', '125', '96', '14', '124', '148', '134', '81', '135', '197', '144', '112', '182', '203', '183', '79', '271', '191', '227', '70', '272', '286', '260', '300', '77', '117'}, {'268', '123', '34', '92', '90', '110', '230', '121', '185', '16', '280', '189', '209', '226', '89', '87', '154', '119', '68', '28', '41', '83', '198', '71', '141'}, {'222', '151', '95', '266', '12', '221', '72', '88', '228', '116', '229', '210', '143', '109', '46', '18', '247', '265', '288', '173', '223', '251', '78', '152', '296'}, {'255', '177', '32', '63', '235', '82', '147', '270', '114', '180', '44', '261', '40', '290', '62', '283', '186', '167', '13', '162', '4', '236'}, {'299', '172', '201', '2', '269', '29', '195', '58', '258', '31', '174', '104', '49', '45', '35', '102', '216', '200', '67', '257', '140', '166'}, {'103', '43', '179', '57', '213', '238', '80', '93', '74', '241', '169', '149', '211', '50', '190', '284', '276', '297', '66'}, {'146', '115', '157', '30',

In [8]:
ari, ami = getAMI_ARI(HG,gt,A)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(A))
print("comm-gt =", len(A_gt))

ARI = 0.7424959362241295
AMI = 0.8376186913015435
comm = 15
comm-gt = 18
